In [62]:
import pandas as pd
import os
import re
import csv
import pdfplumber
path = 'C:/Users/zverk/Desktop/ПДФники'
import pymorphy2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from nltk.stem.snowball import SnowballStemmer
import warnings
warnings.filterwarnings('ignore')

## 1.1 Парсинг данных

In [9]:
comp = os.listdir('C:/Users/zverk/Desktop/ПДФники')

In [10]:
comp

['Магистральные линии связи. Строительство и эксплуатация ВОЛП.pdf',
 'Медицинская оптика.pdf',
 'Метрология.pdf',
 'Мехатроника .pdf',
 'Мобильная робототехника.pdf',
 'Обработка листового металла.pdf',
 'Обслуживание грузовой техники.pdf',
 'Обслуживание и ремонт оборудования релейной защиты и автоматики.pdf',
 'Огранка ювелирных вставок.pdf',
 'Организация строительного производства.pdf',
 'Охрана окружающей среды.pdf',
 'Охрана труда.pdf',
 'Плотницкое дело(юниоры).pdf',
 'Плотницкое дело.pdf',
 'Полимеханика и автоматизация.pdf',
 'Правоохранительная деятельность (Полицейский).pdf',
 'Преподавание музыки в школе.pdf',
 'Производственная сборка изделий авиационной техники.pdf',
 'Производство металлоконструкций.pdf',
 'Промышленный дизайн .pdf',
 'Сметное дело.pdf',
 'Спасательные работы.pdf',
 'Столярное дело.pdf',
 'Туроператорская деятельность.pdf',
 'Укладка напольных покрытий.pdf',
 'Управление локомотивом.pdf',
 'Управление экскаватором.pdf',
 'Холодильная техника и системы к

In [11]:
def parser(filename):
    L=[]
    pdf = pdfplumber.open(filename)
    for i in range(15):
        page = pdf.pages[i]
        try:
            table = page.extract_table()
            df = pd.DataFrame(table)
            if df.shape[0]>1:
                df = df.drop([0,2,5,6,8], axis=1)
                for i in range(len(df)):
                    if df.iloc[i][4] != None:
                        df.loc[i][3] = df.loc[i][4]
                df.drop([4], axis=1, inplace=True)
                L.append(df)
                if "Всего" in table[-1]:
                    break 
        except Exception:
            continue
    df = pd.concat(L)
    df = df.reset_index(drop=True)
    df.columns = ["Раздел", "Слова", "Важность"]
    df = df.drop([0,1])
    return df

In [12]:
DF= []
for comp in comps:
    C = comp[:comp.find(".")]
    filepath = path+"\\"+comp
    try:
        df = parser(filepath)
        df["comp"] = C
        DF.append(df)
    except Exception:
        continue

In [13]:
df = pd.concat(DF)

In [15]:
df = df.reset_index(drop=True)

In [16]:
df

,Раздел,Слова,Важность,comp
0,1,"Организовывать, планировать и анализировать ра...",7,Медицинская оптика
1,None,Специалист должен знать и понимать: \nназначен...,None,Медицинская оптика
2,None,Специалист должен уметь: \nподготавливать и по...,None,Медицинская оптика
3,2,Организовывать сбыт продукции,6,Медицинская оптика
4,None,Специалист должен знать и понимать: \n• основ...,None,Медицинская оптика
...,...,...,...,...
436,None,• оценивать правильность работы электрических...,None,Холодильная техника и системы кондиционирования
437,6,Устранение неисправностей в системах ХС и КВ,20,Холодильная техника и системы кондиционирования
438,None,Специалист должен знать и понимать: \n• элект...,None,Холодильная техника и системы кондиционирования
439,None,Специалист должен уметь: \n• проверять и тест...,None,Холодильная техника и системы кондиционирования


## 1.2 Предобработка данных

In [17]:
data = pd.read_excel('dataset.xlsx', header=None)

In [20]:
data.head(1)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,Разработка компьютерных игр и мультимедийных п...,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,13,Специалист должен знать и понимать: • Принципы...,Программирование на языке С#,27,Специалист должен знать и понимать: • Базовые ...,Unity 3D,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
DATA = pd.DataFrame()
N=data.iloc[0][21]
for i in range(len(data)):
    comp = data.iloc[i][0]
    comp_w = data.iloc[i][1]
    for r,v,s in zip(range(2,28,3),range(3,28,3),range(4,28,3)):
        tmp = {"comp": comp, "comp_w": comp_w, "comp_piece": data.iloc[i][r], "imp": data.iloc[i][v], "piece_w": data.iloc[i][s]}
        DATA = DATA.append(tmp,ignore_index=True)

In [22]:
DATA = DATA.dropna()

In [23]:
DATA.head()

,comp,comp_piece,comp_w,imp,piece_w
0,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,Разработка компьютерных игр и мультимедийных п...,13.0,Специалист должен знать и понимать: • Принципы...
1,Разработка компьютерных игр и мультимедийных п...,Программирование на языке С#,Разработка компьютерных игр и мультимедийных п...,27.0,Специалист должен знать и понимать: • Базовые ...
2,Разработка компьютерных игр и мультимедийных п...,Unity 3D,Разработка компьютерных игр и мультимедийных п...,30.0,Специалист должен знать и понимать: • Основные...
3,Разработка компьютерных игр и мультимедийных п...,Оптимизация компьютерных игр и приложений,Разработка компьютерных игр и мультимедийных п...,10.0,Специалист должен знать и понимать: • Особенно...
4,Разработка компьютерных игр и мультимедийных п...,Анализ и проектирование,Разработка компьютерных игр и мультимедийных п...,15.0,Специалист должен знать и понимать: • Важность...


In [24]:
df["comp_piece"] = None
df.columns = ['comp_w', 'piece_w', 'imp', 'comp', 'comp_piece']

In [25]:
df = pd.concat([DATA, df])

In [26]:
df.shape

(1247, 5)

In [27]:
df.to_csv('preData.csv')

In [29]:
df.head(20)

,comp,comp_piece,comp_w,imp,piece_w
0,Разработка компьютерных игр и мультимедийных п...,Организация и управление работой,Разработка компьютерных игр и мультимедийных п...,13.0,Специалист должен знать и понимать: • Принципы...
1,Разработка компьютерных игр и мультимедийных п...,Программирование на языке С#,Разработка компьютерных игр и мультимедийных п...,27.0,Специалист должен знать и понимать: • Базовые ...
2,Разработка компьютерных игр и мультимедийных п...,Unity 3D,Разработка компьютерных игр и мультимедийных п...,30.0,Специалист должен знать и понимать: • Основные...
3,Разработка компьютерных игр и мультимедийных п...,Оптимизация компьютерных игр и приложений,Разработка компьютерных игр и мультимедийных п...,10.0,Специалист должен знать и понимать: • Особенно...
4,Разработка компьютерных игр и мультимедийных п...,Анализ и проектирование,Разработка компьютерных игр и мультимедийных п...,15.0,Специалист должен знать и понимать: • Важность...
5,Разработка компьютерных игр и мультимедийных п...,Публикация,Разработка компьютерных игр и мультимедийных п...,5.0,Специалист должен знать и понимать: • Особенно...
8,Разработка мобильных приложений(юниоры),Организация и управление работой,Смартфоны продаются и продажи только растут. К...,5.0,Специалист должен знать и понимать: • Принципы...
9,Разработка мобильных приложений(юниоры),"Решение проблем, инновационность и инициативность",Смартфоны продаются и продажи только растут. К...,5.0,Специалист должен знать и понимать: • Общие ти...
10,Разработка мобильных приложений(юниоры),Анализ и проектирование мобильного приложения,Смартфоны продаются и продажи только растут. К...,30.0,Специалист должен знать и понимать: • Важность...
11,Разработка мобильных приложений(юниоры),Разработка мобильных приложений,Смартфоны продаются и продажи только растут. К...,50.0,Специалист должен знать и понимать: • Важность...


## 1.3 Чистка и выделение атрибутов

In [32]:
df = df.dropna(subset=['piece_w'])

In [33]:
df['piece_w'] = df['piece_w'].apply(lambda x: re.sub("[^a-zA-Zа-яА-Я]", " ", x).lower().strip())

In [34]:
stopwords = ["специалист","c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]

In [35]:
df["piece_w"] = df["piece_w"].apply(lambda x: " ".join(word for word in x.split() if word not in stopwords))

In [37]:
morph = pymorphy2.MorphAnalyzer()

In [38]:
df["piece_w"] = df["piece_w"].apply(lambda x: " ".join(morph.parse(word)[0].normal_form for word in x.split()))

In [39]:
print(df.head(1)["piece_w"].values)

['принцип навык обеспечивать продуктивный работа преимущество использование современный средство разработка использовать предоставить ресурс продуктивный работа применять исследовательский технология навык представление самый последний отраслевой рекомендация анализировать результат собственный деятельность сравнение ожидание потребность клиент организация']


In [40]:
df.to_csv('finalDF.csv')

## 1.4 Формирование словарей данных

#### 1. частотный словарь

In [43]:
cv = CountVectorizer(min_df=0.05, max_df=0.25)

In [44]:
cvA = cv.fit_transform(df["piece_w"]).toarray()

In [45]:
sl1 = cv.vocabulary_

In [46]:
sl1 = pd.DataFrame(sl1, index=["Частота"])

In [47]:
sl1

,принцип,навык,использование,современный,средство,разработка,применять,технология,результат,деятельность,...,изделие,размер,ручной,измерение,расчёт,обработка,подготовка,copyright,союз,ворлдскиллс
Частота,76,39,30,101,110,92,75,116,94,16,...,24,91,96,25,93,49,65,0,107,9


#### 2. Векторизированный словарь

In [48]:
cv =CountVectorizer()

In [49]:
cvA = cv.fit_transform(df['piece_w']).toarray()

In [52]:
cvDF = pd.DataFrame(cvA, columns=cv.get_feature_names())

In [53]:
cvDF['T'] = df['comp'].values

In [54]:
cvDF.to_csv('WordsVector.csv')

### Анализ возможности определения блока компетенций

In [55]:
tar = 'Медицинская оптика'
MO = cvDF[cvDF['T'] == tar].drop('T', axis=1)
for i in range(MO.shape[0]):
    for j in cvDF.drop('T', axis=1):
        if MO.iloc[i][j]>2:
            print(j, MO.iloc[i][j])

аксессуар 5
зрение 5
коррекция 5
оптический 5
размещать 3
средство 5
правило 3
средство 3
линза 3
оправа 3
оптический 3
прибор 3
оправа 4
очко 4
ремонт 3
оправа 4
очко 3


In [56]:
tar = 'Преподавание в младших классах'
MO = cvDF[cvDF['T'] == tar].drop('T', axis=1)
for i in range(MO.shape[0]):
    for j in cvDF.drop('T', axis=1):
        if MO.iloc[i][j]>2:
            print(j, MO.iloc[i][j])

средство 3
безопасность 4
возраст 3
правило 3
работа 3
ребёнок 3
соответствие 4
деятельность 4
обеспечивать 3
обучение 4
применять 3
учебный 3
взаимодействие 5
педагогический 3
ребёнок 3
семья 5
форма 3
деятельность 4
профессиональный 3
начальный 4
образование 4
образовательный 3
обучение 3
общий 4
педагогический 3
предмет 3
технология 3


#### стеминг

In [63]:
stemmer = SnowballStemmer("russian") 
def stemming_tokenizer(text):
    text = re.split('\W+', text)
    text = [stemmer.stem(word) for word in text]
    return text

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1246 entries, 0 to 440
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comp        1246 non-null   object
 1   comp_piece  806 non-null    object
 2   comp_w      910 non-null    object
 3   imp         910 non-null    object
 4   piece_w     1246 non-null   object
dtypes: object(5)
memory usage: 58.4+ KB
